# Lấy thông tin từng ngôi nhà

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
import string
import unidecode
import csv
import re # getting numbers from string

In [26]:
#support method:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def remove_accent(text):
    return unidecode.unidecode(text)

def convert_price(value, unit):
    units = {
        'ty': 9,
        'trieu': 6
    }
    # remove accent vietnamese
    unit = remove_accent(unit)
    try:
        exponential = units[unit]
        value_number = float(value)
        price = int(value_number) * pow(10, exponential)
        return price
    except ValueError:
        return 0
    except:
        return 0

In [27]:
class House:
    #attributes:
    def __init__(self):    
        self.__id = ""
        self.__price = 0 # VND
        self.__area = 0 # Đơn vị: m2
        self.__floorCount = 1
        self.__roomCount = 0
        self.__bedRoomCount = 0
        self.__toiletCount = 0
        self.__furniture = "" #
        self.__facade = 0
        self.__view = ""
        self.__direction = 0 # 0: Non-determined, 1: Bac, 2: DB, 3:Dong, 4:DN, 5:Nam, 6:TN, 7:Tay, 8:TB}
        self.__distanceToMainRoad = 0 # Đơn vị: m
        self.__ownershipCertification = ""
        self.__landOwnershipCertification = 0 #False
        self.__houseOwnershipCertification = 0 #False
        self.__location = ""
        self.__type = ""
#         self.__type = 0 #0: Nhà riêng, 1: dự án
    
    # Set attribute function
    def __set_id(self, value):
        self.__id = value
        
    def __set_price(self, value):
        self.__price = value
        
    def __set_landOwnershipCertificate(self, value):
        self.__landOwnershipCertificate = value
        
    def __set_houseOwnershipCertification(self, value):
        self.__houseOwnershipCertification = value
        
    def __set_id_from_string(self, string):
        #process string
        id = string
        self.__set_id(id)
                   
    def __set_price_from_string(self, string):
        price = 0;
        if is_number(string):
            price = int(string)
        else:
            x = string.split()
            value = x[0]
            unit = x[len(x)-1]
            if is_number(value):
                price = convert_price(value, unit)
        self.__set_price(price)
    
    def set_area_from_string(self, string):   
        temp = re.findall(r'\d+', string)
        self.__area = temp[0]
        
    def set_roomCount_from_string(self, string):
        temp = re.findall(r'\d+', string)
        self.__roomCount = temp[0]
        
    def set_type_from_string(self, string):
        temp = string.split('\n')
        self.__type = temp[1]
    
    def set_location_from_string(self, string):
        temp = string.split('\n')
        self.__location = temp[1]
        
    def set_direction_from_string(self, string):
        if string == 'Bắc': 
            self.__direction = 1 
            return
        if string == 'Đông Bắc': 
            self.__direction = 2 
            return
        if string == 'Đông': 
            self.__direction = 3 
            return
        if string == 'Đông Nam': 
            self.__direction = 4 
            return
        if string == 'Nam': 
            self.__direction = 5 
            return
        if string == 'Tây Nam': 
            self.__direction = 6 
            return
        if string == 'Tây': 
            self.__direction = 7 
            return
        if string == 'Tây Bắc': 
            self.__direction = 8 
            return 
        self.__direction = 0 
        return
    
    def __set_floorCount_from_string(self, string):
        temp = re.findall(r'\d+', string)
        self.__floorCount = temp[0]
        
    def __set_furniture_from_string(self, string):
        self.__furniture = string.split("\n")[1]
        
    def __set_ownershipCertification_from_string(self, string):
        self.__ownershipCertification = string.split("\n")[1]
        if ("sổ hồng" in string) or ("Sổ hồng" in string):
            self.__set_landOwnershipCertificate(0)
            self.__set_houseOwnershipCertification(1)
        elif ("sổ đỏ" in string) or ("Sổ đỏ" in string):
            self.__set_landOwnershipCertificate(1)
            self.__set_houseOwnershipCertification(0)
            
    def __set_bedRoomCount_from_string(self, string):
        temp = re.findall(r'\d+', string)
        self.__bedRoomCount = temp[0]
        
#     def set_direction_from_string(self, string):
#         self.__direction = string
        
    def set_attribute(self, keyword, string):
        if keyword == 'id':
            self.__set_id_from_string(string)
            return
        if keyword == 'price':
            self.__set_price_from_string(string)
            return
        if keyword == 'area':
            self.set_area_from_string(string)
            return
        if keyword == 'roomcount':
            self.set_roomCount_from_string(string)
            return
        if keyword == 'type':
            self.set_type_from_string(string)
            return
        if keyword == 'location':
            self.set_location_from_string(string)
            return
        if keyword == 'direction':
            self.set_direction_from_string(string)
            return
        
        if keyword == 'floorCount':
            self.__set_floorCount_from_string(string)
        
        if keyword == 'furniture':
            self.__set_furniture_from_string(string)
        
        if keyword == 'ownershipCertification':
            self.__set_ownershipCertification_from_string(string)
        
        if keyword == 'bedRoomCount':
            self.__set_bedRoomCount_from_string(string)
        
    #Get function
#     def __get_attribute(self, keyword)

    def get_id(self):
        return self.__id
    
    def get_price(self):
        return self.__price
    
    def get_area(self):
        return self.__area
    
    def get_floorCount(self):
        return self.__floorCount
    
    def get_roomCount(self):
        return self.__roomCount
    
    def get_bedRoomCount(self):
        return self.__bedRoomCount
    
    def get_toiletCount(self):
        return self.__toiletCount
    
    def get_furniture(self):
        return self.__furniture
    
    def get_facade(self):
        return self.__facade
    
    def get_view(self):
        return self.__view
    
    def get_direction(self):
        return self.__direction
    
    def get_distanceToMainRoad(self):
        return self.__distanceToMainRoad
    
    def get_ownershipCertification(self):
        return self.__ownershipCertification
    
    def get_landOwnershipCertification(self):
        return self.__landOwnershipCertification
    
    def get_houseOwnershipCertification(self):
        return self.__houseOwnershipCertification
    
    def get_location(self):
        return self.__location
    
    def get_type(self):
        return self.__type
        
    
    def __del__(self):
        self.__id = None
        self.__price = None
        self.__area = None
        self.__floorCount = None
        self.__roomCount = None
        self.__bedRoomCount = None
        self.__toiletCount = None
        self.__furniture = None
        self.__facade = None
        self.__view = None
        self.__direction = None
        self = None
        self.__ownershipCertification = None
        self.__landOwnershipCertification = None
        self.__houseOwnershipCertification = None
        self.__location = None
        self.__type = None
        
    #---------------print object house--------------
    def __str__(self):
        return "id = %s\n" \
            "price = %d\n" \
            "area = %s\n" \
            "roomCount = %s\n" \
            "bedRoomCount = %s\n" \
            "toiletCount = %s\n" \
            "floorCount = %s\n" \
            "furniture = %s\n" \
            "facade = %s\n" \
            "view = %s\n" \
            "direction = %s\n" \
            "distanceToMainRoad = %s\n" \
            "ownershipCertification = %s\n" \
            "landOwnershipCertification = %s\n" \
            "houseOwnershipCertification = %s\n" \
            "location = %s\n" \
            "type = %s"% (
                self.__id, self.__price, self.__area, self.__roomCount, 
                self.__bedRoomCount, self.__toiletCount, self.__floorCount, self.__furniture,
                self.__facade, self.__view,
                self.__direction, self.__distanceToMainRoad, self.__ownershipCertification,
                self.__landOwnershipCertification,
                self.__houseOwnershipCertification, self.__location, self.__type)
        
    #public method

In [28]:
def get_detail_first(house):
    wait = WebDriverWait(browser, 2000)
    element = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'short-detail-2')))
    short_detail_2 = browser.find_element_by_class_name('short-detail-2')
    short_detail_2_values = short_detail_2.find_elements_by_tag_name('li')
    for value in short_detail_2_values:
        if 'giá' in value.text:
            price = value.find_elements_by_tag_name('span')[1].text
            house.set_attribute('price', price)
        if 'Diện tích' in value.text:
            house.set_attribute('area', value.text)
        if 'Phòng ngủ' in value.text:
            house.set_attribute('roomcount', value.text)

In [29]:
def get_detail_second(house):
    detail_2 = browser.find_element_by_class_name('detail-2')
    detail_2_values = detail_2.find_element_by_class_name('box-round-grey3').find_elements_by_tag_name('div')

    for value in detail_2_values:
        if 'Loại' in value.text:
            house.set_attribute('type', value.text)
        if 'Địa chỉ' in value.text:
            house.set_attribute('location', value.text)
        if 'Hướng' in value.text:
            house.set_attribute('direction', value.text)
        if 'Số tầng' in value.text:
            house.set_attribute('floorCount', value.text)
        if 'Nội thất' in value.text:
            house.set_attribute('furniture', value.text)
        if 'Pháp lý' in value.text:
            house.set_attribute('ownershipCertification', value.text)
        if 'phòng ngủ' in value.text:
            house.set_attribute('bedRoomCount', value.text)

In [30]:
def init_out_file_header():
    header = [
        'ID',
    #     'Giá'
        'Price',
    #     'Diện tích',
        'Area',
    #     Số tầng
        'floorCount',
    #     'Số Phòng',
        'RoomCount',
    #     'Số Phòng Ngủ'
        'BedRoomCount',
    #     'Số Phòng VS'
        'ToiletCount',
    #     Mức độ hoàn thiện đồ nội thất, đơn vị: %
        'Furniture',
    #     'Mặt Tiền'
        'Facade',
    #     Khung cảnh
        'View',
    #     'Hướng nhà'
        'Direction',
    #     'khoảng cách đến trục đường chính'
        'DistanceToMainRoad',
    #     Pháp lý
        'OwnershipCertification',
    #     'Sổ đỏ'
        'LandOwnershipCertification',
    #     'Sổ hồng'
        'houseOwnershipCertification',
        #Vị trí, dia chi
        'Location',
        # Project type
        'Type',
    #     now: string
    #     'DatNhaRieng': 0 after processing
    #     'DatDuAn': 1
    ]
    return header

In [31]:
def get_all_house_information(n, outFilePath):
    count = 500;
    
    outF = open(outFilePath, 'w', encoding='utf-8')
    header = init_out_file_header()
    writer = csv.DictWriter(outF, fieldnames=header)
    
    with outF:
        writer.writeheader()
    
        while count < n:
            print(count, '\n')
            count += 1
            inFilePath = 'Data' + '\\' + 'house_links_' + str(count) + '.txt'
            inF = open(inFilePath, 'r', encoding='utf-8')
            lines = inF.readlines()
            inF.close()

            stt = 0;
            for line in lines:
                stt += 1
                url = line
                browser.get(url)
                #FID: file id
                #STTIF: stt in file
                hid = 'FID' + str(count) + 'STTIF' + str(stt)

                house = House()
                house.set_attribute('id', hid)

                get_detail_first(house)
                get_detail_second(house)
                
                writer.writerow({
                    'ID': house.get_id(),
                    'Price': house.get_price(),
                    'Area': house.get_area(),
                    'floorCount': house.get_floorCount(),
                    'RoomCount': house.get_roomCount(),
                    'BedRoomCount': house.get_bedRoomCount(),
                    'ToiletCount': house.get_toiletCount(),
                    'Furniture': house.get_furniture(),
                    'Facade': house.get_facade(),
                    'View': house.get_view(),
                    'Direction': house.get_direction(),
                    'DistanceToMainRoad': house.get_distanceToMainRoad(),
                    'OwnershipCertification': house.get_ownershipCertification(),
                    'LandOwnershipCertification': house.get_landOwnershipCertification(),
                    'houseOwnershipCertification': house.get_houseOwnershipCertification(),
                    'Location': house.get_location(),
                    'Type': house.get_type(),
                })
                print (stt)
                
    outF.close()

In [32]:
browser = webdriver.Chrome(executable_path='chromedriver.exe')
get_all_house_information(1000, 'Data\house_information.csv')


500 

1


Exception ignored in: <function House.__del__ at 0x00000213575B1550>
Traceback (most recent call last):
  File "<ipython-input-27-0e9bb8cdab4c>", line 223, in __del__
AttributeError: 'NoneType' object has no attribute '_House__ownershipCertification'


2


Exception ignored in: <function House.__del__ at 0x00000213575B1550>
Traceback (most recent call last):
  File "<ipython-input-27-0e9bb8cdab4c>", line 223, in __del__
AttributeError: 'NoneType' object has no attribute '_House__ownershipCertification'


3


...

Exception ignored in: <function House.__del__ at 0x00000213575B1550>
Traceback (most recent call last):
  File "<ipython-input-27-0e9bb8cdab4c>", line 223, in __del__
AttributeError: 'NoneType' object has no attribute '_House__ownershipCertification'


20
524 



Exception ignored in: <function House.__del__ at 0x00000213575B1550>
Traceback (most recent call last):
  File "<ipython-input-27-0e9bb8cdab4c>", line 223, in __del__
AttributeError: 'NoneType' object has no attribute '_House__ownershipCertification'


1


KeyboardInterrupt: 